# Recommendation Engine using Restricted Boltzmann Machines
Example adapted from: 
- https://github.com/echen/restricted-boltzmann-machines
- https://github.com/AmirAli5/Deep-Learning

Data obtained from:
- https://www.kaggle.com/datasets/eswarchandt/amazon-movie-ratings

## Boltzmann Machines


## Data Preprocessing

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

## Amazon data test

In [ ]:
# Test Amazon movie recommendation set
amazon_df = pd.read_csv('Amazon.csv')
amazon_df.head()

In [ ]:
# Transform 1-5 scale to binary
amazon_df = amazon_df.applymap(lambda x: 0 if (isinstance(x, float) and x<=3) else x)
amazon_df = amazon_df.applymap(lambda x: 1 if (isinstance(x, float) and x>3) else x)
amazon_df = amazon_df.replace(np.NaN,0)
amazon_df

In [ ]:
# Format the data for pytorch
amazon_df.drop(columns='user_id', inplace=True)

In [ ]:
# Select a random subset of columns for training
train, test = train_test_split(amazon_df, test_size = 0.2)
n_users = np.shape(train)[1]

In [ ]:
# Convert the data into torch tensor
training_set = torch.FloatTensor(train.to_numpy())
test_set = torch.FloatTensor(test.to_numpy())

## Building Restricted Boltzmann Machine

In [ ]:
# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = n_users
batch_size = 1
rbm = RBM(nv, nh)

## Train the Restricted Boltzmann Machine

In [ ]:
# Training the RBM model
nb_epoch = 10
loss = []
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, n_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    loss.append(float(train_loss/s))
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

## Test the Restricted Boltzmann Machine

In [ ]:
# Testing the RBM Model
test_loss = 0
s = 0.
for id_user in range(n_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))